In [1]:
import os
import sys
 

current = os.getcwd()
 
parent = os.path.dirname(current)
 
sys.path.append(parent)


import torch
from torch.autograd import Variable
from mpc import mpc
from mpc.mpc import QuadCost, LinDx

In [2]:
torch.manual_seed(0)

n_batch, n_state, n_ctrl, T = 1, 2, 3, 4
n_sc = n_state + n_ctrl

# Randomly initialize a PSD quadratic cost and linear dynamics.
C = torch.randn(T*n_batch, n_sc, n_sc)
C = torch.bmm(C, C.transpose(1, 2)).view(T, n_batch, n_sc, n_sc)
"""
for i in range(T):
    for j in range(n_batch):
        for k in range(n_sc):
            C[i,j, k, n_sc-1] = C[i,j, n_sc-1, k] = 0.
"""
print("shape C", C, C.shape, C.requires_grad)
c = torch.randn(T, n_batch, n_sc)

alpha = 0.2
R = (torch.eye(n_state)+alpha*torch.randn(n_state, n_state)).repeat(T, n_batch, 1, 1)
S = torch.randn(T, n_batch, n_state, n_ctrl)
F = torch.cat((R, S), dim=3)


print("F shape ", F.shape, F.requires_grad)

# The initial state.
x_init = torch.randn(n_batch, n_state)

# The upper and lower control bounds.
u_lower = -torch.rand(T, n_batch, n_ctrl)
u_upper = torch.rand(T, n_batch, n_ctrl)

x_lqr, u_lqr, objs_lqr = mpc.MPC(
    n_state=n_state,
    n_ctrl=n_ctrl,
    T=T,
    u_lower=u_lower, 
    u_upper=u_upper,
    lqr_iter=20,
    verbose=1,
    backprop=False,
    exit_unconverged=False,
)(x_init, QuadCost(C, c), LinDx(F))

print(u_lqr)

shape C tensor([[[[ 3.5668, -1.0969, -0.3683,  2.6897,  1.0361],
          [-1.0969,  6.7528, -1.1207,  3.0238,  1.2773],
          [-0.3683, -1.1207,  3.0108,  0.8809,  2.8207],
          [ 2.6897,  3.0238,  0.8809,  5.7177,  4.1796],
          [ 1.0361,  1.2773,  2.8207,  4.1796,  6.8899]]],


        [[[ 4.8533, -1.2580,  1.3891,  0.7717, -0.9349],
          [-1.2580,  2.2275, -0.4020, -0.8436,  3.9401],
          [ 1.3891, -0.4020,  0.8729,  0.5548, -0.2986],
          [ 0.7717, -0.8436,  0.5548,  0.9985, -0.5189],
          [-0.9349,  3.9401, -0.2986, -0.5189, 11.1903]]],


        [[[ 1.6471,  0.5893, -2.3478,  2.8452, -0.1097],
          [ 0.5893, 14.8537, -0.2706,  6.6078, -3.6476],
          [-2.3478, -0.2706,  7.8072, -1.0785, -2.0783],
          [ 2.8452,  6.6078, -1.0785, 12.5197, -2.5834],
          [-0.1097, -3.6476, -2.0783, -2.5834,  3.5236]]],


        [[[ 3.3002, -3.7269,  2.2901,  0.6564,  0.1191],
          [-3.7269,  8.0130, -3.1425, -2.1537, -0.1012],
          [

/home/carlos/anaconda3/envs/mpc_pytorch_test/lib/python3.8/site-packages/torch/_tensor.py:738: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_pivots) (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1992.)
  LU, pivots, infos = torch._lu_with_info(
/home/carlos/Desktop/Austin/SeungHyeonProject/mpc.pytorch/mpc/pnqp.py:31: UserWarning: torch.lu_solve is deprecated in favor of torch.linalg.lu_solveand will be removed in a future PyTorch release.
Note that torch.linalg.lu_solve has its arguments reversed.
X = torch.lu_solve(B, LU, pivots)
should be replaced with
X = torch.linalg.lu_solve(LU, pivots, B) (Tr